In [1]:
#Import packages

import requests
import time
import urllib
from lxml import etree
import os
import sqlite3
import urllib.parse
import urllib.request
import lxml
import urllib.error
import lxml.html
from lxml import etree
from urllib.request import urlopen
from IPython.core.display import display, HTML
import json
import sys
from urllib.parse import quote
from urllib.request import urlopen
from urllib.error import HTTPError, URLError


#Additional function

def clean(string: str) -> str:
    
    #Clean input string and URL encode
    
    #Note: the function does NOT lowercase the string! The reason is that upon inspecting the MARC21 code, author appeared
    #      to be capitalized for the HPB database
    
    string = urllib.parse.quote(string)
    return string

In [8]:
#Establishing connection with the local sqlite database

conn = sqlite3.connect('stcv.sqlite')

def reconciliator(gen_conn):

#Function that takes an squilte3 connection:
#Performs a query of 100 book titles from an stcv database and collects title, author, and yop inside a dictionary

#Through a HPB’s SRU/CQL query, it reads  the results from the website in MARC21
#Executes two queries:
#if author, title, and yop are available inside the stcv database, uses all of them to return the first 10 matches
#if some of the values (e.g., only 13 out of the 100 records have a missing value, and the exception query triggers
#only in that case) are missing, queries only by title

#It does not return an object: it prints out the matches for each c:stcv with HPB, who the author is for that record. what's the year of publication,
#and compares them to the searched author and yop

#Note: there is not comparison for title search: this comparison is available only for the second query. No titles were matching
#      due to the diffences in spelling

#Note: U have bettered the code to include author and yop for each record in the first query to asses the quality of match
#      for each match. This causes 14 records to not be match. If that part of the code is erased, the code will match all
#      the records. Between a program that returns 84 matches with additional info about the match quality and one that returns
#      100 matches with no information about any of those matches, I have chosen the former.
    
    
    conn = gen_conn
    c = conn.cursor()

    #query first 100 volumes in the stcv database

    query = """
    SELECT DISTINCT
        title.cloi as identifier,
        COUNT(title.cloi) as id_count,
        author_vw as author_standardized,
        author_zvwr as author_original,
        author_zbd as author_dates,
        corporateauthor_nm as corporateauthor_standardized,
        corporateauthor_zvwr as corporateauthor_original,
        title_ti as title_title,
        title_lg as title_language,
        collation_fm as format,
        collation_ka as quires,
        collation_pg as pages,
        edition_ed as edition_info,
        impressum_ju1sv as year1,
        impressum_ju1ty as year1_type,
        impressum_ju2sv as year2,
        impressum_ju2ty as year2_type,
        impressum_pl as place,
        impressum_ug as printer,
        language_lg as language_info,
        number_nr as fingerprint
        FROM title
    LEFT JOIN author on author.cloi = title.cloi
    LEFT JOIN collation on collation.cloi = title.cloi
    LEFT JOIN corporateauthor on corporateauthor.cloi = title.cloi
    LEFT JOIN edition on edition.cloi = title.cloi
    LEFT JOIN impressum on impressum.cloi = title.cloi
    LEFT JOIN language on language.cloi = title.cloi
    LEFT JOIN number on number.cloi = title.cloi
    GROUP BY identifier
    HAVING ID_COUNT=2
    LIMIT 100
    """


    stcv_d = {} #dictionary for final result


    #execute query
    c.execute(query)
    data = [row for row in c.fetchall()]

    #Append results
    for row in data:
        
        #Appending title, author, and yop to a dictionary having c:stcv
        
        stcv_d[row[0]] = [row[7], row[2], row[13]]

    #Close file
    
        conn.close()
    
    #Tags dictionary for xml query in MARC21
    
    ns = {
    'zs': "http://docs.oasis-open.org/ns/search-ws/sruResponse",
    'diag': "http://docs.oasis-open.org/ns/search-ws/diagnostic",
    'slim': "http://www.loc.gov/MARC21/slim",}
    
    # HPB MARC21 sru basic url

    HPB = "https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query="    
    
    #iterate over metadata dictionary
    
    for key, value in stcv_d.items():
        
        title = value[0] #Obtain title
        author = value[1] #Obtain author
        year = value[2]  #Obtain year of publication
        
        HPB_record_counter = 0 #Counter used to keep track of the record match number (maximumRecords=10)
        
        try:
            print('(Querying the databse with our best metadata...)', '\n'*2)
            
            #First query: author, title and year, with starting record = 1 and maximumRecords=10
            
            url = HPB + f"{clean(author)}%20\"{clean(title)}\"%20{year}" + "&startRecord=1&maximumRecords=10"
            query = etree.parse(urllib.request.urlopen(url)) #Parsing the url into a xml.etree.ElementTree object
            
            #Querying the element
            root = query
            #Searching only in the current sub-tree (.) for all the <numberOfRecords elements
            num_records = int(root.find(".//zs:numberOfRecords", namespaces=ns).text)
            
            #Searching only in the current sub-tree (.) for all the <records>
            records = root.findall(".//zs:record", namespaces=ns)
            
            #Loop over the records to find specific subelements
            for r, record in enumerate(records):
                HPB_record_counter += 1 #Update counter for each record matching a specific c:stvc identifier
                
                #datafield_tag35: HPB identifier
                datafield_tag35 = record.find(".//slim:datafield[@tag='035']", namespaces=ns)
                datafield_tag35_a = datafield_tag35.find('slim:*[@code="a"]', namespaces=ns)
                
                #datafield_tag100: author for that specific record
                datafield_tag100 = record.find(".//slim:datafield[@tag='100']", namespaces=ns)
                datafield_tag100_a = datafield_tag100.find('slim:*[@code="a"]', namespaces=ns)
                
                #datafield_tag264: yop for that specific record
                datafield_tag264 = record.find(".//slim:datafield[@tag='264']", namespaces=ns)
                datafield_tag264_c = datafield_tag264.find('slim:*[@code="c"]', namespaces=ns)
                
                #Pritning matches
                print("- The STCV identifier", key, "corresponds to the following HPB identifier", str(int(r)+1), "=>",
                        datafield_tag35_a.text, ", which is record match number", HPB_record_counter,
                        "for this c:stcv code.", '\n'*2,
                #Author retrieved for the record vs. author searched
                      
                        "1. The author is: ", datafield_tag100_a.text, '\n'*2,
                        "   The author you looked for is: ",author, '\n'*2,
                      
                #Yop retrieved for the record vs. author searched
                        "2. The yop is: ", datafield_tag264_c.text, '\n'*2,
                        "   The yop you looked for is: ", year, '\n'*2,
                        f"This info can be retrieved at the following url {url}",
                        '\n')
                
                #Goodness of match (note: title never perfectly matches, so it was excluded)
                
                #Perfect match (author and yop are the same)
                if datafield_tag100_a.text==author and datafield_tag264_c.text==year:
                    print('The above match is a perfect match!', '\n'*3)
                #Normal match (at least one between author and yop are the same)
                elif datafield_tag264_c.text!=year or datafield_tag100_a.text!=author:
                    print('The above match is a normal match!', '\n'*3)
                    
                #Bad match (at least one between author and yop are the same)
                elif datafield_tag264_c.text!=year and datafield_tag100_a.text!=author:
                    print('The above match is not a good match!', '\n'*3 )


        #The exception triggers when one of the values (not tile! is 0)
        
        except Exception:
            
            #Try with querying by title only
            
            try:
                print('(Could not find anything. Querying the databse per title...)', '\n'*2)
                
                HPB_record_counter = 0 #Counter used to keep track of the record match number (max#New url: accordint to imumRecords=10)
                
                #New url: accordint to HPB index, title are searched with pica.tit (or db.tit)
                
                new_url = f"https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=pica.tit={clean(title)}&startRecord=1&maximumRecords=10"
                new_query = etree.parse(urllib.request.urlopen(new_url))
                root = new_query
                
                #Querying all the records
                new_records = root.findall(".//zs:record", namespaces=ns)
                
                #Iterating over each record
                for r, record in enumerate(new_records):
                    HPB_record_counter += 1
                    
                    #datafield_tag35: HPB identifier
                    datafield_tag35 = record.find(".//slim:datafield[@tag='035']", namespaces=ns)
                    datafield_tag35_a = datafield_tag35.find('slim:*[@code="a"]', namespaces=ns)
                    
                    #datafield_tag245: title
                    datafield_tag245 = record.find(".//slim:datafield[@tag='245']", namespaces=ns)
                    datafield_tag245_a = datafield_tag245.find('slim:*[@code="a"]', namespaces=ns)
                    
                    #Record printing
                    print("- The STCV identifier", key, "corresponds to the following HPB identifier", str(int(r)+1), "=>",
                            datafield_tag35_a.text, ", which is record match number", HPB_record_counter,
                            "for this c:stcv code.", '\n'*2,
                          
                            #Title comparison
                            f"1. We looked for this title: {title}", '\n'*2,
                            f"   And got this title back: {datafield_tag245_a.text[0:50]}", '\n'*2,
                          
                            #url
                            f"This info can be retrieved at the following url {new_url}",
                            '\n'*2)
                #If the query founds no record...
                
                if len(new_records)<1:
                    print('Stop: This also failed. Sorry, we really could not match your record', '\n'*2)
                    
                    #I have trie to add further code to query by author, but it does not find anything and causes the last
                    #exception to trigger
                    
            except Exception:
                print('WARNING: Fatal error in our query!')

In [9]:
reconciliator(conn)

(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12854444 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12854444 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Ernst, Simon-Pierre 

    The author you looked for is:  Ernst, Simon-Pierre 

 2. The yop is:  1797 

    The yop you looked for is:  1797 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Ernst%2C%20Simon-Pierre%20"Apologie%20des%20ministres%20du%20culte%2C%20qui%20ont%20pr%C3%AAt%C3%A9%20la%20d%C3%A9claration%20exig%C3%A9e%20par%20la%20loi%20du%207%20vend%C3%A9m.%20an%204.%20contre%20les%20critiques%20de%20mm.%20D%C3%A9doyar%20%26%20Vanhoren%2C%20les%20Motifs%20de%20Malines%20%26%20autres%20brochures"%201797&startRecord=1&maximumRecords=10 

The above match is a perfect match! 



(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12854501 corresponds 

- The STCV identifier c:stcv:12857052 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12857052 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Courvoisier, Jean-Jacques 

    The author you looked for is:  Courvoisier, Jean-Jacques 

 2. The yop is:  1642 

    The yop you looked for is:  1642 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Courvoisier%2C%20Jean-Jacques%20"Le%20prince%20immortel%20tir%C3%A9%20sur%20la%20vie%20%26%20la%20fin%20glorieuse%20de%20son%20altesse%20royale%20don%20Ferdinand%20d%27Avstriche"%201642&startRecord=1&maximumRecords=10 

The above match is a perfect match! 



(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12857069 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12857069 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Choquet, François Hyacinthe 

    The autho

- The STCV identifier c:stcv:12858392 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12858392 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Drexelius, Hieremias 

    The author you looked for is:  Drexelius, Hieremias 

 2. The yop is:  1642 

    The yop you looked for is:  1642 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Drexelius%2C%20Hieremias%20"Tobias%20morali%20doctrin%C3%A2%20illustratus%20%5B...%5D%20ex%20posthumis%20libellis%20quartus.%20Monachii%20M.DC.XLI."%201642&startRecord=1&maximumRecords=10 

The above match is a perfect match! 



(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12858408 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12858408 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Drexelius, Hieremias 

    The author you looked for is:  Drexelius, Hieremias 

 2

- The STCV identifier c:stcv:12859250 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12859250 , which is record match number 1 for this c:stcv code. 

 1. The author is:  van der Linden, Jan 

    The author you looked for is:  Linden, van der, Jan 

 2. The yop is:  inter[[1687-1794]-YYYY] 

    The yop you looked for is:  1687 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Linden%2C%20van%20der%2C%20Jan%20"Heerlycke%20ende%20gheluckighe%20reyse%20na%20het%20Heyligh%20Landt%20ende%20de%20stadt%20van%20Jerusalem%20%5B...%5D.%20Het%20eerste%20deel"%201687&startRecord=1&maximumRecords=10 

The above match is a normal match! 



(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12859427 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12859427 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Faber, Matthias 

    The author you lo

- The STCV identifier c:stcv:12859871 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:7088533 , which is record match number 1 for this c:stcv code. 

 1. The author is:  de Limon, Geoffroi 

    The author you looked for is:  Limon, de, Geoffroi 

 2. The yop is:  1793 

    The yop you looked for is:  1793 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Limon%2C%20de%2C%20Geoffroi%20"La%20vie%20et%20le%20martyre%20de%20Louis%20XVI%20%5B...%5D%20immol%C3%A9%20le%2021%20janvier%201793%3B%20avec%20un%20examen%20du%20d%C3%A9cret%20r%C3%A9gicide"%201793&startRecord=1&maximumRecords=10 

The above match is a normal match! 



- The STCV identifier c:stcv:12859871 corresponds to the following HPB identifier 2 => BE-AnVE.c:stcv:12923154 , which is record match number 2 for this c:stcv code. 

 1. The author is:  de Limon, Geoffroi 

    The author you looked for is:  Limon, de, Geoffroi 

 2. The yop is:  17

...

- The STCV identifier c:stcv:12868640 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12868640 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Henschenius, Godefridus 

    The author you looked for is:  Henschenius, Godefridus 

 2. The yop is:  1653 

    The yop you looked for is:  1653 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Henschenius%2C%20Godefridus%20"De%20episcopatv%20Traiectensi%20%5B...%5D%20diatriba"%201653&startRecord=1&maximumRecords=10 

The above match is a perfect match! 



(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12868663 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12868663 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Alexander VII. 

    The author you looked for is:  Alexander VII [Pope] 

 2. The yop is:  1654 

    The yop you looked for is:  1654 

 Th

- The STCV identifier c:stcv:12869479 corresponds to the following HPB identifier 1 => US-icn.01.365209 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Favart 

    The author you looked for is:  Favart, Charles-Simon 

 2. The yop is:  1819-20 

    The yop you looked for is:  1763 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Favart%2C%20Charles-Simon%20"l%27Anglois%20a%20Bordeaux%3B%20comedie%20en%20un%20acte%20et%20en%20vers%20libres"%201763&startRecord=1&maximumRecords=10 

The above match is a normal match! 



- The STCV identifier c:stcv:12869479 corresponds to the following HPB identifier 2 => FR-751131015.CG.FRBNF304222880000000 , which is record match number 2 for this c:stcv code. 

 1. The author is:  Favart, Charles-Simon 

    The author you looked for is:  Favart, Charles-Simon 

 2. The yop is:  1788 

    The yop you looked for is:  1763 

 This info can be retrieved at

- The STCV identifier c:stcv:12869588 corresponds to the following HPB identifier 1 => BE-KBR00.CAT.1_1780441 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Sedaine, Michel Jean 

    The author you looked for is:  Sedaine, Michel Jean 

 2. The yop is:  1812 

    The yop you looked for is:  1764 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=Sedaine%2C%20Michel%20Jean%20"Rose%20et%20Colas%2C%20com%C3%A9die%20en%20un%20acte"%201764&startRecord=1&maximumRecords=10 

The above match is a normal match! 



- The STCV identifier c:stcv:12869588 corresponds to the following HPB identifier 2 => FR-751131015.CG.FRBNF399705600000005 , which is record match number 2 for this c:stcv code. 

 1. The author is:  Sedaine, Michel-Jean 

    The author you looked for is:  Sedaine, Michel Jean 

 2. The yop is:  1781 

    The yop you looked for is:  1764 

 This info can be retrieved at the following 

- The STCV identifier c:stcv:12869812 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12869812 , which is record match number 1 for this c:stcv code. 

 1. We looked for this title: De indvciis avt pace Belgarvm dissertationes variæ 

    And got this title back: De indvciis avt pace Belgarvm dissertationes variæ 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=pica.tit=De%20indvciis%20avt%20pace%20Belgarvm%20dissertationes%20vari%C3%A6&startRecord=1&maximumRecords=10 


- The STCV identifier c:stcv:12869812 corresponds to the following HPB identifier 2 => DE-601.GVK.489072526 , which is record match number 2 for this c:stcv code. 

 1. We looked for this title: De indvciis avt pace Belgarvm dissertationes variæ 

    And got this title back: De Indvciis Avt Pace Belgarvm Dissertationes Variæ 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&q

- The STCV identifier c:stcv:12870054 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12870054 , which is record match number 1 for this c:stcv code. 

 1. We looked for this title: Catechisme ov Doctrine chrestienne divisée en cinq parties, et quarante & vne leçons 

    And got this title back: Catechisme ov Doctrine chrestienne divisée en cinq 

 This info can be retrieved at the following url https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query=pica.tit=Catechisme%20ov%20Doctrine%20chrestienne%20divis%C3%A9e%20en%20cinq%20parties%2C%20et%20quarante%20%26%20vne%20le%C3%A7ons&startRecord=1&maximumRecords=10 


(Querying the databse with our best metadata...) 


- The STCV identifier c:stcv:12870121 corresponds to the following HPB identifier 1 => BE-AnVE.c:stcv:12870121 , which is record match number 1 for this c:stcv code. 

 1. The author is:  Drexelius, Hieremias 

    The author you looked for is:  Drexelius, Hieremias 

 2. The yop is:  1651 

    Th